#### 데이터셋 다운로드
- 데이터 URL 사용
    * urllib 패키지 활용 : from urllib.request import urlretrieve
    * 함수 :

In [1]:
### 모듈 로딩
from urllib.request import urlretrieve
import os

### 파이토치의 내장 이미지 데이터 셋
from torchvision import datasets

In [2]:
LOGO_URL = 'https://www.google.co.kr/images/branding/googlelogo/2x/googlelogo_color_272x92dp.png'
PATH = '../image/'
FILE_NAME = '../image/logo.png'

In [3]:
# 데이터 저장 폴더 존재 여부 체크 후 생성
if not os.path.exists(PATH):
    os.mkdir(PATH)

In [4]:
# 데이터 다운로드 후 이미지 저장
urlretrieve(LOGO_URL,FILE_NAME)

('../image/logo.png', <http.client.HTTPMessage at 0x29f8d858b20>)

In [5]:
### MNIST 데이터셋 다운로드
datasets.MNIST('../data/',download=True)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data/
    Split: Train

- 바이너리 데이터 읽기

In [6]:
# 데이터 파일리스트 준비
MNIST_PATH = '../data/MNIST/raw/'
filelist = os.listdir(MNIST_PATH)
for file in filelist:
    if file.endswith('.gz'):
        filelist.remove(file)
print(f'filelist => {filelist}')

filelist => ['t10k-images-idx3-ubyte', 't10k-labels-idx1-ubyte', 'train-images-idx3-ubyte', 'train-labels-idx1-ubyte']


In [7]:
with open(MNIST_PATH+filelist[0],mode='rb') as f:
    print(f.read(50))

b"\x00\x00\x08\x03\x00\x00'\x10\x00\x00\x00\x1c\x00\x00\x00\x1c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"


In [8]:
# 데이터 파일 읽기

# 오류 1
# with open(filelist[0]) as f:
#     pass
# => 이렇게 하면 못찾음. 파일명만 있고 경로는 없어서

# 오류 2
# with open(MNIST_PATH+filelist[0],encoding='utf-8') as f:
# #                           utf-8은 텍스트인데 텍스트가 아니기때문에 읽을 수가 없음.
#     data =f.read(50)
#     print(data)

# 답
with open(MNIST_PATH+filelist[0],mode='rb') as f:
    data =f.read(4)
    print(data)
    
    data =f.read(4)
    print(data)    

b'\x00\x00\x08\x03'
b"\x00\x00'\x10"


- 바이너리 데이터를 원하는 자료형으로 읽기
    * 파이썬 내장 모듈 struct 사용

In [9]:
# 모듈 로딩
import struct

In [10]:
with open(MNIST_PATH+filelist[0],mode='rb') as f:
    data = f.read(8)
    
    print(struct.unpack('>II',data)) # I -> 'unsigned int', Standard size=4
    #                                   https://docs.python.org/3/library/struct.html => 여기에 I 정보 있음.
    # (2051, 10000) => 2051 : 매직코드(앞에 4개), 10000 : 데이터 개수
    print(struct.unpack('<II',data))

#

(2051, 10000)
(50855936, 270991360)


In [11]:
# 데이터 파일 읽어서 데이터셋 파일 생성
def makeDataFile(imgfile,labelfile,datasetfile,dataLength=None):
    with open(imgfile,mode='rb') as data_f:
        with open(labelfile,mode='rb') as label_f:
            with open(datasetfile,mode='w') as dataset_f:
        # 파일의 헤더 정보 읽기
        
                _,img_cnt = struct.unpack('>II',data_f.read(8))
                rows,cols = struct.unpack('>II',data_f.read(8))

                _,_ = struct.unpack('>II',label_f.read(8))
                # if dataLength : 
                for n in range(img_cnt):
                    # 이미지 raw 데이터
                    pixels = data_f.read(rows*cols)
                    sdata=list(map(lambda n:str(n),pixels))
                    #print(sdata)

                    # 이미지 라벨
                    label=struct.unpack('B',label_f.read(1))[0]
                    #print(label)

                    # 피쳐와 라벨 연결
                    sdata.insert(0,str(label))
                    #print(sdata)

                    # 데이터 파일에 기록
                    img_label=','.join(sdata) # 디버깅 용도
                    dataset_f.write(img_label+'\n')
                    #print(img_label)
    

In [12]:
DS_PATH = '../data/'
MNIST_PATH = '../data/MNIST/raw/'
TRAIN_FILE = DS_PATH+'MNIST_train.csv'
TEST_FILE = DS_PATH+'MNIST_test.csv'


makeDataFile(MNIST_PATH+filelist[0],MNIST_PATH+filelist[1],TEST_FILE)
makeDataFile(MNIST_PATH+filelist[2],MNIST_PATH+filelist[3],TRAIN_FILE)